In [ ]:
!pip install tiktoken
!pip install langchain
!pip install openai

# Embeddings con OpenAI

Hay varios modelos que son capaces de hacer un embedding de nuestro texto, en este cuaderno estaremos utilizando el embedding de OpenAI, el cual tiene la capacidad de posicionas muy bien palabras o textos segun su semantica. Este es el mismo embedding que utilizan los modelos gpt. En este cuaderno estarás haciendo embedding de un texto para despues poder buscar cosas dentro de este texto por medio de preguntas en lenguaje natural.

[Embeddings - OpenAI API](https://platform.openai.com/docs/guides/embeddings/what-are-embeddings)

In [1]:
# Importamos todas las dependencias requeridas
import openai
import pandas as pd # Utilizaremso pandas para la creacion de dataframes que nos servira como una seudo-base de datos
from openai import OpenAI

client = OpenAI()

# Miselanea

## Calcular costos de un embedding

In [2]:
def print_embeddings_cost(texts):
  import tiktoken
  enc = tiktoken.encoding_for_model('text-embedding-ada-002')
  total_tokens = sum([len(enc.encode(page)) for page in texts])
  print(f'Total Tokens: {total_tokens}')
  print(f'Embeddings consto en dolar:  {total_tokens / 1000 * 0.0001:.5f}')

## [Calculo de simulitudes](https://ashukumar27.medium.com/similarity-functions-in-python-aa6dfe721035)

### cosine_similarity

Para encontrar la similitudes entre dos vectores tenemos que hacer una operacion llamado [``cosine_similarity``](https://en.wikipedia.org/wiki/Cosine_similarity) o similitud del coseno es una medida de la similitud entre dos vectores no nulos definidos en un espacio con producto interno, es el producto escalar de los vectores dividido por el producto de sus longitudes, la similitud del coseno siempre pertenece al intervalo [-1, 1].

In [3]:
import numpy as np
from numpy.linalg import norm
# Funcion para obtener las similitudes entre vectores
def cosine_similarity(A, B):
    similitud = np.sum(A*B)/(norm(A)*norm(B))
    similitud = max(min(similitud, 1), -1)
    return similitud

### euclidean_similarity

La euclidean_similarity o similitud euclidiana es una medida de la similitud entre dos vectores basada en la distancia euclidiana. La distancia euclidiana entre dos vectores X e Y se define como la raíz cuadrada de la suma de las diferencias al cuadrado entre los elementos correspondientes de los dos vectores:

La similitud euclidiana se puede obtener restando la distancia euclidiana de una constante mayor que la distancia máxima posible. Por ejemplo, si los vectores tienen valores entre 0 y 1, la constante podría ser 1. Así, la similitud euclidiana entre X e Y sería:

s(X,Y)=1−d(X,Y)

In [4]:
def euclidean_similarity(vector1, vector2):
    # Asegúrate de que ambos vectores tengan la misma longitud
    if len(vector1) != len(vector2):
        raise ValueError("Los vectores deben tener la misma longitud")

    # Calcula la distancia euclidiana entre los dos vectores
    distance = np.linalg.norm(np.array(vector1) - np.array(vector2))

    # La similitud es el inverso de la distancia (mientras más pequeña la distancia, más similar)
    similarity = 1 / (1 + distance)

    return similarity

# Que es y cómo usar embeddings
Al hacer embedding de un dato, lo estamos convirtiendo a un vector numérico, datos similares estarán más cercanos entre si cuando semanticamente son similares

In [5]:
# Creamos la funcion para crear embeddings
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding


## Embeddings de palabras

In [6]:
# Se puede hacer embeeding de palabras o cadenas de texto
palabras = ["casa", "perro", "gato", "lobo", "leon", "zebra", "tigre"]

In [7]:
# Calculamos sus costo
print_embeddings_cost(list(palabras))

Total Tokens: 14
Embeddings consto en dolar:  0.00000


In [8]:
diccionario = {}
for i in palabras:
    diccionario[i] = get_embedding(i, model="text-embedding-ada-002")
    

## Operacioners con los embeddings

In [10]:
palabra = "gato"
print("Primeros 10 valores de {}:\n".format(palabra), diccionario[palabra][:10])
print("\n")
print("Número de dimensiones del dato embebido\n", len(diccionario[palabra]))


Primeros 10 valores de gato:
 [-0.024525104090571404, 0.009590277448296547, -0.00033545654150657356, -0.011137725785374641, -0.007165075279772282, 0.011865937151014805, -0.022041384130716324, -0.024941224604845047, -0.005227514076977968, -0.018972495570778847]


Número de dimensiones del dato embebido
 1536


## Comparar dos embeddings
Debido a que los embeddings son una representacion vectorial de los datos en un espacio latente, podemos medir la distancia entre dos vectores y asi obtener que tan similares son. Podemos comparar una palabra nueva o alguna de las que ya fueron embebidas 
OJO: No necesariamente es similitud al objeto. Ej. perro y gato aun siendo "opuestos" semanticamente estan cerca pues tienen una relación.

In [11]:
import numpy as np

n_palabra = "lobo" # Palabra nueva a comparar
palabra_comparar = "perro" # Palabra del diccionario con la que compararemos la nueva palabra
n_palabra_embed = get_embedding(n_palabra, model="text-embedding-ada-002")
similitud_euclidiana = euclidean_similarity(np.array(diccionario[palabra_comparar]), np.array(n_palabra_embed))
similitud_coseno = cosine_similarity(np.array(diccionario[palabra_comparar]), np.array(n_palabra_embed))
print("Similitud ueclidina entre {} y {} es: {}".format(n_palabra, palabra_comparar, similitud_euclidiana))
print("Similitud coseno entre {} y {} es: {}".format(n_palabra, palabra_comparar, similitud_coseno))

Similitud ueclidina entre lobo y perro es: 0.6495839328206657
Similitud coseno entre lobo y perro es: 0.8544985060169356


# Sumar embeddings
Como los vectores contienen valores numericos, podemos sumarlos y el resultado será un nuevo vector de un concepto que una los elementos sumados

In [46]:
from numpy.linalg import norm

# Suma los dataframes correctamente
sumados = pd.DataFrame(diccionario["leon"]) + pd.DataFrame(diccionario["zebra"])

# Normalizamos el vector y que quitamos una dimecion 
sumados_normalized = np.squeeze(np.array((sumados / norm(sumados))))

for key, value in diccionario.items():
    # Convierte a array de NumPy antes de calcular la similitud
    vector_normalized = np.array(diccionario[key]) / norm(diccionario[key]) # Normaliza el vector
    result_cosemo = cosine_similarity(vector_normalized, sumados_normalized)
    result_eu = euclidean_similarity(vector_normalized, sumados_normalized)
    print(f"similitud cose de {key}: {result_cosemo}")
    print(f"similitud euclidiana de {key}: {result_eu}")


similitud cose de casa: 0.8277789592555793
similitud euclidiana de casa: 0.6301626539054387
similitud cose de perro: 0.8503498226441224
similitud euclidiana de perro: 0.6463775230907934
similitud cose de gato: 0.8561199380195257
similitud euclidiana de gato: 0.6508582836418143
similitud cose de lobo: 0.8563832328729764
similitud euclidiana de lobo: 0.6510663671129061
similitud cose de leon: 0.9534578304124934
similitud euclidiana de leon: 0.7662264179447055
similitud cose de zebra: 0.9534578298521325
similitud euclidiana de zebra: 0.7662264168663956
similitud cose de tigre: 0.8804617125440589
similitud euclidiana de tigre: 0.6716121618627884


# Aplicacion de un Chatbot

Usaremos Gradio para hacer una interfaz básica donde podremos hacer preguntas y obtendremos una respuesta. 
Para esto reutilizaremos lo que hemos visto hasta el momento pero usaremos el archivo de **chatbot_qa.csv**

In [13]:
# Funcion para crear el embeddion de nuestro texto
def embed_text(path="texto.csv"):
    conocimiento_df = pd.read_csv(path)
    print_embeddings_cost(conocimiento_df["texto"]) # calculamos el costo
    conocimiento_df['Embedding'] = conocimiento_df['texto'].apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))
    conocimiento_df.to_csv('embeddings.csv')
    return conocimiento_df

texto_emb = embed_text("./Doc/chatbot_qa.csv")

Total Tokens: 84
Embeddings consto en dolar:  0.00001


In [54]:
import ast
# Funcion para agendar cita
def buscar(busqueda, datos, n_resultados=5):
    # embedding de la busqueda
    print_embeddings_cost(busqueda)
    busqueda_embed = get_embedding(busqueda, model="text-embedding-ada-002")
    # Convertimos los campos de embedding en listas
    datos['Embedding'] = datos['Embedding'].apply(ast.literal_eval)
    # Creamos un campo para las euclidean_similarity, cosine_similarity y aplicamos la operacion a los valores del embeddinng
    datos["Similitud_coseno"] = datos['Embedding'].apply(lambda x: cosine_similarity(np.array(x), np.array(busqueda_embed)))
    datos["Similitud_eu"] = datos['Embedding'].apply(lambda x: euclidean_similarity(np.array(x), np.array(busqueda_embed)))
    # Ordenamos de mayor a menor por similitud
    datos_cos = datos.sort_values("Similitud_coseno", ascending=False)
    datos_eu = datos.sort_values("Similitud_eu", ascending=False)
    # Retornamos un dataframe con las primeras 5 respuestas
    return [datos_cos.iloc[:n_resultados][["texto", "Similitud_coseno", "Embedding"]], datos_eu.iloc[:n_resultados][["texto", "Similitud_eu", "Embedding"]] ]

[coseno, eucliden] = buscar('Cuales son los servicios?', pd.read_csv('./embeddings.csv'))

Total Tokens: 25
Embeddings consto en dolar:  0.00000


## Similitud cosine_similarity

In [55]:
coseno

,texto,Similitud_coseno,Embedding
3,Contamos con servicios de carpinteria y herrería,0.814882,"[-0.0018300488591194153, 0.009386846795678139,..."
4,No contamos con servicio de jardinería,0.797549,"[-0.00400054594501853, -0.0003356921370141208,..."
2,Las citas se pueden agendar al: 523 432 4212,0.789926,"[-0.01759340986609459, -0.007559466175734997, ..."
1,Estamos abiertos de Lunes a Sabado de 9 a 16:00,0.779414,"[-0.01700492762029171, -0.0039050509221851826,..."
5,Las cotizaciones se hacen una vez que ya se de...,0.772910,"[0.0038105526473373175, -2.1932393792667426e-0..."


## Similitud euclidean_similarity


In [56]:
eucliden

,texto,Similitud_eu,Embedding
3,Contamos con servicios de carpinteria y herrería,0.621709,"[-0.0018300488591194153, 0.009386846795678139,..."
4,No contamos con servicio de jardinería,0.611128,"[-0.00400054594501853, -0.0003356921370141208,..."
2,Las citas se pueden agendar al: 523 432 4212,0.606727,"[-0.01759340986609459, -0.007559466175734997, ..."
1,Estamos abiertos de Lunes a Sabado de 9 a 16:00,0.600886,"[-0.01700492762029171, -0.0039050509221851826,..."
5,Las cotizaciones se hacen una vez que ya se de...,0.597397,"[0.0038105526473373175, -2.1932393792667426e-0..."


# Procesar datos de un PDF
Haremos ahora un ejemplo donde leemos un PDF para poder hacer preguntas y traer un exctracto del PDF

In [ ]:
# Paquete para leer el pdf
!pip install pypdf

## Carga del documento PDF

In [16]:
# pip install langchain pypdf
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

loader = PyPDFLoader("./Doc/mtg.pdf")
pages = loader.load_and_split()

In [17]:
# Un elemento por cada página
pages[3].page_content

'Jasper SandnerC 214/ 269 2/1Oo2\n™ & © 2014 Wizards of the CoastSP• M15Campo de \nbatallatú \n16 vidas\nrestantes\nyo\n18 vidas \nrestantesCementerio  Biblioteca  \nBiblioteca  Mano\nManoCementerio  \n3Para comenzar el juego, baraja tu mazo, \ntambién conocido como tu biblioteca. Roba una mano de siete cartas y comprueba cuántas tierras tienes. Puedes mirar la línea de texto que hay bajo la ilustración de cada carta para ver de qué tipo de carta se trata. Para este primer juego, si no tienes al menos dos tierras, baraja de nuevo tu mazo (incluyendo tu mano anterior) y roba una mano nueva.\nCada jugador comienza con 20 vidas, \ny cada uno debe llevar la cuenta de su total de vidas de alguna manera (con un dado, lápiz y papel...). ¡Reduce el total de vidas de tu oponente a 0 y ganarás el juego!Comenzar'

In [18]:
# Objeto que va a hacer los cortes en el texto
split = CharacterTextSplitter(chunk_size=300, separator = '.\n')

In [19]:
textos = split.split_documents(pages) # Lista de textos

Created a chunk of size 364, which is longer than the specified 300
Created a chunk of size 326, which is longer than the specified 300
Created a chunk of size 325, which is longer than the specified 300
Created a chunk of size 503, which is longer than the specified 300
Created a chunk of size 1507, which is longer than the specified 300
Created a chunk of size 308, which is longer than the specified 300
Created a chunk of size 583, which is longer than the specified 300
Created a chunk of size 458, which is longer than the specified 300
Created a chunk of size 429, which is longer than the specified 300
Created a chunk of size 626, which is longer than the specified 300
Created a chunk of size 358, which is longer than the specified 300
Created a chunk of size 892, which is longer than the specified 300
Created a chunk of size 1311, which is longer than the specified 300
Created a chunk of size 333, which is longer than the specified 300
Created a chunk of size 1139, which is longer 

In [20]:
print(textos[0].page_content)
print(textos[0])

Guía de inicio 
rápidoEdad: 13 o +
page_content='Guía de inicio \nrápidoEdad: 13 o +' metadata={'source': './Doc/mtg.pdf', 'page': 0}


## Guardamos los parramos en un dataframe

In [21]:
# Extraemos la parte de page_content de cada texto y lo pasamos a un dataframe
textos = [str(i.page_content) for i in textos] #Lista de parrafos
parrafos = pd.DataFrame(textos, columns=["texto"])
print(parrafos)

                                                 texto
0                  Guía de inicio \nrápidoEdad: 13 o +
1    2Bienvenido a Magic: The Gathering, el mejor j...
2    Si ya tienes algunas cartas de Magic, es el mo...
3    Hay miles de cartas para elegir, por lo \nque ...
4    Siempre que ganes vidas, puedes \nponer un con...
..                                                 ...
107  ∙Paso de mantenimiento\n ∙Paso de robar: roba ...
108  Fase de combate\n ∙Paso de inicio del combate\...
109  ∙ Paso de declarar bloqueadoras: cada criatura...
110  ∙ Paso de daño de combate: las criaturas bloqu...
111  ∙Paso de final del combate.\nFase principal (d...

[112 rows x 1 columns]


# Creamos los emmbeddings

In [24]:
# Calculamos el costo de los embeddings
print_embeddings_cost(parrafos["texto"])

Total Tokens: 11769
Embeddings consto en dolar:  0.00118


In [25]:
parrafos['Embedding'] = parrafos["texto"].apply(lambda x: get_embedding(x, model='text-embedding-ada-002')) # Nueva columna con los embeddings de los parrafos
parrafos.to_csv('./Doc/MTG.csv')

In [34]:
def buscar(busqueda, datos, n_resultados=5):
    # embedding de la busqueda
    print_embeddings_cost(busqueda) # calculamos el costo
    busqueda_embed = get_embedding(busqueda, model="text-embedding-ada-002")
    # Convertimos los campos de embedding en listas
    datos['Embedding'] = datos['Embedding'].apply(ast.literal_eval)
    # Creamos un campo para el cosine_similarity y aplicamos la operacion a los valores del embeddinng
    datos["Similitud"] = datos['Embedding'].apply(lambda x: cosine_similarity(np.array(x), np.array(busqueda_embed)))
    # Ordenamos de mayor a menor por similitud
    datos = datos.sort_values("Similitud", ascending=False)
    # Retornamos un dataframe con las primeras 5 respuestas
    return datos.iloc[:n_resultados][["texto"]].astype(str)


buscar("Con cuanta vida empiezo?", pd.read_csv('./Doc/MTG.csv'), 5) # Reutilizamos la funcion de "buscar" del app de gradio

Total Tokens: 24
Embeddings consto en dolar:  0.00000


,texto
15,"Cada jugador comienza con 20 vidas, \ny cada u..."
4,"Siempre que ganes vidas, puedes \nponer un con..."
14,Jasper SandnerC 214/ 269 2/1Oo2\n™ & © 2014 Wi...
24,Cuando el Favor divino entre al \ncampo de bat...
12,Cuando el Favor divino entre al \ncampo de bat...
